In [ ]:
!pip install nibabel scikit-image

In [ ]:
import torch
from torch.utils.data import Subset
import numpy as np
import os
from zipfile import ZipFile
from PIL import Image
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from zipfile import ZipFile

# Ruta al archivo zip en Drive
zip_path = '/content/drive/My Drive/TFM/MSLesSeg-Dataset.zip'

# Ruta donde quieres extraerlo
extract_path = '/content/drive/My Drive/TFM/MSLesSeg-Dataset/'

# Extraer
with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


KeyboardInterrupt: 

In [ ]:
import nibabel as nib
import numpy as np
from pathlib import Path
from skimage import io
from tqdm import tqdm

def to_uint8(data):
    data -= data.min()
    ptp = np.ptp(data)
    if ptp == 0:
        return np.zeros(data.shape, dtype=np.uint8)
    data /= ptp
    data *= 255
    return data.astype(np.uint8)

def nii_to_jpgs(input_path, output_dir, rgb=False):
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    data = nib.load(input_path).get_fdata()
    num_slices, width, height = data.shape
    nombre_archivo = input_path.stem
    nombre_base = nombre_archivo.replace('_MASK.nii', '')
    #print(nombre_base)
    for slice in range(num_slices):
        slice_data = data[slice, ...]
        slice_data = to_uint8(slice_data)
        if rgb:
            slice_data = np.stack(3 * [slice_data], axis=2)
        output_path = output_dir / f'{nombre_base}_slice_{slice}.png'
        #print(output_path)
        io.imsave(output_path, slice_data)

def process_training_set(input_root, output_root, rgb=False):
    input_root = Path(input_root)
    output_root = Path(output_root)

    nii_files = list(input_root.rglob('*MASK.nii.gz'))

    print(f"Se encontraron {len(nii_files)} archivos .nii.gz en {input_root}")

    for nii_file in tqdm(nii_files, desc='Procesando archivos'):
        # Ruta relativa desde input_root
        relative_path = nii_file.relative_to(input_root)
        # Sacar nombre de archivo sin extensión .nii.gz
        nii_name = nii_file.stem
        # Crear carpeta de salida para este archivo
        #output_dir = output_root / relative_path.parent / nii_name
        #output_dir.mkdir(parents=True, exist_ok=True)
        # Procesar el archivo
        nii_to_jpgs(nii_file, output_root, rgb=rgb)


In [ ]:
input_path_train = '/content/drive/My Drive/TFM/MSLesSeg-Dataset/train'
output_dir_train = '/content/drive/My Drive/TFM/MSLesSeg-Dataset/output_trainMasks'
input_path_test = '/content/drive/My Drive/TFM/MSLesSeg-Dataset/test'
output_dir_test = '/content/drive/My Drive/TFM/MSLesSeg-Dataset/output_test'
process_training_set(input_path_train, output_dir_train, rgb=True)
#process_training_set(input_path_test, output_dir_test, rgb=True)

Se encontraron 92 archivos .nii.gz en /content/drive/My Drive/TFM/MSLesSeg-Dataset/train


Procesando archivos: 100%|██████████| 92/92 [06:19<00:00,  4.12s/it]


In [ ]:
import os
import cv2
from pathlib import Path
from tqdm import tqdm

def combine_modalities(flair_dir, t1_dir, t2_dir, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)

    flair_slices = sorted(flair_dir.glob("*.png"))
    for flair_path in flair_slices:
        fname = flair_path.name
        t1_path = t1_dir / fname
        t2_path = t2_dir / fname

        if not (t1_path.exists() and t2_path.exists()):
            print(f"Slice {fname} faltante en alguna modalidad, se omite.")
            continue

        flair = cv2.imread(str(flair_path), cv2.IMREAD_GRAYSCALE)
        t1 = cv2.imread(str(t1_path), cv2.IMREAD_GRAYSCALE)
        t2 = cv2.imread(str(t2_path), cv2.IMREAD_GRAYSCALE)

        if flair is None or t1 is None or t2 is None:
            print(f"Error al cargar slice {fname}")
            continue

        rgb = cv2.merge([flair, t1, t2])
        cv2.imwrite(str(output_dir / fname), rgb)

def process_dataset(root_input, root_output):
    root_input = Path(root_input)
    root_output = Path(root_output)

    print(root_output)

    patients = list(root_input.glob("P*"))

    print(patients)

    for patient_path in tqdm(patients, desc=f"Procesando {root_input.name}"):
        scans = list(patient_path.glob("P*_T*"))  # T1, T2, T3...
        print(scans)
        for scan_path in scans:
            scan_name = f"{patient_path.name}_{scan_path.name}"

            flair_dir = scan_path / f"{scan_name}_FLAIR.nii"
            t1_dir    = scan_path / f"{scan_name}_T1.nii"
            t2_dir    = scan_path / f"{scan_name}_T2.nii"

            if not (flair_dir.exists() and t1_dir.exists() and t2_dir.exists()):
                print(f"Faltan modalidades en {scan_path}")
                continue

            output_dir = root_output / patient_path.name / scan_path.name
            combine_modalities(flair_dir, t1_dir, t2_dir, output_dir)


In [ ]:
input_base = Path("/content/drive/My Drive/TFM/MSLesSeg-Dataset")
output_base = input_base / "combined_rgb"

#process_dataset(input_base / "output_train", output_base / "train")
process_dataset(input_base / "output_test", output_base / "test")

In [ ]:
import cv2
from pathlib import Path
from tqdm import tqdm

def combine_slices_png(flair_dir, t1_dir, t2_dir, output_dir):
    output_dir.mkdir(parents=True, exist_ok=True)
    flair_slices = sorted(flair_dir.glob("*.png"))

    for flair_path in flair_slices:
        fname = flair_path.name
        t1_path = t1_dir / fname
        t2_path = t2_dir / fname

        if not (t1_path.exists() and t2_path.exists()):
            print(f"Slice {fname} faltante en alguna modalidad")
            continue

        flair = cv2.imread(str(flair_path), cv2.IMREAD_GRAYSCALE)
        t1 = cv2.imread(str(t1_path), cv2.IMREAD_GRAYSCALE)
        t2 = cv2.imread(str(t2_path), cv2.IMREAD_GRAYSCALE)

        if flair is None or t1 is None or t2 is None:
            print(f"Error al leer {fname}")
            continue

        rgb = cv2.merge([flair, t1, t2])
        cv2.imwrite(str(output_dir / fname), rgb)

def process_test_from_png(input_root, output_root):
    input_root = Path(input_root)
    output_root = Path(output_root)

    for patient_dir in tqdm(sorted(input_root.glob("P*")), desc="Procesando test set"):
        patient_id = patient_dir.name
        flair_dir = patient_dir / f"{patient_id}_FLAIR.nii"
        t1_dir = patient_dir / f"{patient_id}_T1.nii"
        t2_dir = patient_dir / f"{patient_id}_T2.nii"

        if not (flair_dir.exists() and t1_dir.exists() and t2_dir.exists()):
            print(f"Modalidades faltantes en {patient_id}")
            continue

        out_dir = output_root / patient_id
        combine_slices_png(flair_dir, t1_dir, t2_dir, out_dir)

In [ ]:
input_test_path = Path("/content/drive/My Drive/TFM/MSLesSeg-Dataset/output_test")
output_test_path = Path("/content/drive/My Drive/TFM/MSLesSeg-Dataset/combined_rgb/test")

process_test_from_png(input_test_path, output_test_path)

Procesando test set: 100%|██████████| 22/22 [47:08<00:00, 128.57s/it]
